In [10]:
# dit moet je wel weghalen bij inleveren, die staat nu ook in de yml file...
#pip install imblearn

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report

from sklearn.feature_selection import SelectKBest, chi2

from sklearn.decomposition import PCA

from sklearn.linear_model import Lasso

from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE, RFECV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

from pandas.plotting import scatter_matrix 
from sklearn.metrics import confusion_matrix

## load tested data


In [12]:
# Load the data from the .csv file
data = pd.read_csv('cleaned_descriptor_data',index_col=0)

# Scale the data with a standard scaling
scaling=MinMaxScaler()
scaling.fit(data)
scaled_data=scaling.transform(data)
df_data = pd.DataFrame(scaled_data, columns=data.columns,index=data.index)

# Choose the kinase of which you want to predict the inhibition
#selected_kinase = 'ERK2_inhibition'
selected_kinase = 'PKM2_inhibition'    

# Define train and test data
X = df_data.drop(columns=['ERK2_inhibition','PKM2_inhibition']).copy()
y = data[selected_kinase].copy()


In [13]:
X_fingerprints=pd.read_csv('fingerprint_data',index_col=0)

## Load untested data

In [14]:
untested_molecules=pd.read_csv('cleaned_descriptor_data_untested',index_col=0)
untested_molecules.head()
X_untested=untested_molecules.drop(columns=['ERK2_inhibition','PKM2_inhibition']).copy()
y_untested=untested_molecules[selected_kinase].copy()


In [15]:
X_untested_fingerprints=pd.read_csv('fingerprint_data_untested',index_col=0)

## Recursive Feature Exclusion

In [16]:
rfe = RFE(RandomForestClassifier(), n_features_to_select=50)
rfe.fit(X, y)
selected_features = X.columns[rfe.support_]
print(selected_features)

Index(['MaxAbsEStateIndex', 'MinAbsEStateIndex', 'MinEStateIndex', 'qed',
       'MinPartialCharge', 'FpDensityMorgan1', 'FpDensityMorgan2',
       'FpDensityMorgan3', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI',
       'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI',
       'BCUT2D_MRLOW', 'AvgIpc', 'BalabanJ', 'BertzCT', 'Chi1v', 'Chi3n',
       'Chi3v', 'HallKierAlpha', 'Ipc', 'Kappa1', 'PEOE_VSA10', 'PEOE_VSA3',
       'PEOE_VSA5', 'PEOE_VSA8', 'PEOE_VSA9', 'SMR_VSA1', 'SMR_VSA10',
       'SMR_VSA3', 'SMR_VSA7', 'SMR_VSA9', 'SlogP_VSA2', 'SlogP_VSA3',
       'SlogP_VSA5', 'EState_VSA3', 'EState_VSA6', 'VSA_EState1',
       'VSA_EState2', 'VSA_EState4', 'VSA_EState5', 'VSA_EState6',
       'VSA_EState9', 'NumAromaticHeterocycles', 'MolLogP', 'fr_Ar_N',
       'fr_NH0'],
      dtype='object')


In [17]:
X_cleaned=X[selected_features]
X_untested_cleaned=X_untested[selected_features]

# Random forests

## Random forest with descriptors evaluation

### Find n_estimators

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y, test_size=0.2, stratify=y,random_state=11)


pipeline = imbpipeline(steps = [['smote', SMOTE(random_state=11)],['classifier', RandomForestClassifier()]])

stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=11)
    
param_grid = {'classifier__n_estimators':[100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]}
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring='recall', cv=stratified_kfold, n_jobs=None)

grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
cv_score = grid_search.best_score_
test_score = grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score}\nTest score: {test_score}')


# Get the best estimator from GridSearchCV
best_model = grid_search.best_estimator_

# Predict on test set
y_pred = best_model.predict(X_test)

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(cm)
print(classification_report(y_pred, y_test))

{'classifier__n_estimators': 100}
Cross-validation score: 0.09
Test score: 0.0
Confusion Matrix:
[[218   0]
 [  6   0]]
              precision    recall  f1-score   support

           0       1.00      0.97      0.99       224
           1       0.00      0.00      0.00         0

    accuracy                           0.97       224
   macro avg       0.50      0.49      0.49       224
weighted avg       1.00      0.97      0.99       224



C:\Users\20223692\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\20223692\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\20223692\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y, test_size=0.2)

# Apply SMOTE to balance the training set
smote = SMOTE(random_state=40)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Calculate the ratio between 0 and 1 in the balanced training set and the test set
ratio_train_balanced = y_train_balanced.value_counts(normalize=True)
ratio_test = y_test.value_counts(normalize=True)

print("Ratio of 0 and 1 inhibitors in the balanced training set:")
print(ratio_train_balanced)
print("\nRatio of 0 and 1 inhibitors in the testing set:")
print(ratio_test)

# Train the Random Forest model with the balanced training set
rf_descriptors = RandomForestClassifier(n_estimators=600, random_state=42)

# Perform cross-validation
cv = StratifiedKFold(n_splits=2, shuffle=False)
cv_scores = cross_val_score(rf_descriptors, X_train_balanced, y_train_balanced, cv=cv, scoring='recall')

print(f'Cross-Validation Scores: {cv_scores}')

rf_descriptors.fit(X_train_balanced, y_train_balanced)
y_pred = rf_descriptors.predict(X_test)

# Evaluate the model using a confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_mat)
print(classification_report(y_pred, y_test))


Ratio of 0 and 1 inhibitors in the balanced training set:
0    0.5
1    0.5
Name: PKM2_inhibition, dtype: float64

Ratio of 0 and 1 inhibitors in the testing set:
0    0.96875
1    0.03125
Name: PKM2_inhibition, dtype: float64
Cross-Validation Scores: [1. 1.]
Confusion Matrix:
[[215   2]
 [  7   0]]
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       222
           1       0.00      0.00      0.00         2

    accuracy                           0.96       224
   macro avg       0.50      0.48      0.49       224
weighted avg       0.98      0.96      0.97       224



## Random forest with fingerprints evaluation

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_fingerprints, y, test_size=0.2)

# Apply SMOTE to balance the training set
smote = SMOTE(random_state=40)
X_train_balanced_finger, y_train_balanced_finger = smote.fit_resample(X_train, y_train)

# Calculate the ratio between 0 and 1 in the balanced training set and the test set
ratio_train_balanced_finger = y_train_balanced_finger.value_counts(normalize=True)
ratio_test_finger = y_test.value_counts(normalize=True)

print("Ratio of 0 and 1 inhibitors in the training set:")
print(ratio_train_balanced_finger)
print("\nRatio of 0 and 1 inhibitors in the testing set:")
print(ratio_test_finger)

rf_fingerprints= RandomForestClassifier(n_estimators=600, random_state=40)

# Perform cross-validation
cv = StratifiedKFold(n_splits=2, shuffle=False)
cv_scores = cross_val_score(rf_fingerprints, X_train_balanced_finger, y_train_balanced_finger, cv=cv, scoring='recall')

print(f'Cross-Validation Scores: {cv_scores}')

rf_fingerprints.fit(X_train, y_train)
y_pred = rf_fingerprints.predict(X_test)

conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)
print(classification_report(y_pred, y_test))

Ratio of 0 and 1 inhibitors in the training set:
0    0.5
1    0.5
Name: PKM2_inhibition, dtype: float64

Ratio of 0 and 1 inhibitors in the testing set:
0    0.977679
1    0.022321
Name: PKM2_inhibition, dtype: float64
Cross-Validation Scores: [0.97695853 1.        ]
[[219   0]
 [  5   0]]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       224
           1       0.00      0.00      0.00         0

    accuracy                           0.98       224
   macro avg       0.50      0.49      0.49       224
weighted avg       1.00      0.98      0.99       224



C:\Users\20223692\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\20223692\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\20223692\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Random forest with descriptors on actual data

In [21]:
rf_descriptors = RandomForestClassifier(n_estimators=600)
rf_descriptors.fit(X_cleaned, y)
y_pred_descriptor = rf_descriptors.predict(X_untested_cleaned)
df_y_pred_descriptor=pd.DataFrame(y_pred_descriptor,index=X.index)


## Random forest with fingerprints on actual data

In [22]:
rf_fingerprints = RandomForestClassifier(n_estimators=600)
rf_fingerprints.fit(X_fingerprints, y)
y_pred_fingerprint = rf_fingerprints.predict(X_untested_cleaned)
df_y_pred_fingerprint=pd.DataFrame(y_pred_fingerprint,index=X.index)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- AvgIpc
- BCUT2D_CHGHI
- BCUT2D_CHGLO
- BCUT2D_LOGPHI
- BCUT2D_LOGPLOW
- ...
Feature names seen at fit time, yet now missing:
- Bit_1
- Bit_10
- Bit_100
- Bit_1000
- Bit_1001
- ...


## Intersect

In [ ]:
intersect=X[(df_y_pred_descriptor[0]==1)&(df_y_pred_fingerprint[0]==1)].index
intersect

Index(['C=CCNC(=O)CCCC(=O)NCC=C', 'C=CCOn1c(=O)c(C)[n+]([O-])c2ccccc21',
       'C=CCn1cc(Cl)c(=O)n(CC=C)c1=O', 'CC(C)(N[O-])/C(=N/O)c1cccs1',
       'CC(C)C(=O)Nc1cc(=O)nc2nc[nH]n12', 'CC(C)[NH+](Cc1nc2ccccc2[nH]1)C(C)C',
       'CCC(CC)C(=O)NCc1ccccc1OC', 'CCN(CC)S(=O)(=O)c1cccc2nonc12',
       'CCOC(=O)N=[S@@](N)(=O)c1ccc(Cl)cc1', 'CCc1nnc(NC(=O)C2CCC2)s1',
       ...
       'COc1ccc(OC)c(Nc2cc(-c3ccccc3)nc(N)n2)c1',
       'Cc1nnc(SCCC(=O)Nc2nc(-c3ccccc3)ns2)s1',
       'Cc1noc(C)c1CSc1nc2[nH]ncc2c(=O)n1-c1ccccc1',
       'O=C(COc1ccc(Cl)cc1)Nc1nc(-c2cccs2)cs1',
       'O=C1OCC2=C1[C@@H](c1ccc3c(c1)OCO3)Sc1ccccc1N2',
       'O=C1c2ccccc2[C@H](Nc2ccc3c(c2)OCCO3)N1Cc1ccco1',
       'O=S(=O)(Nc1cccc(-c2cn3ccsc3[nH+]2)c1)c1ccc(F)cc1',
       'Oc1c(C[NH+]2CCN(c3ccccn3)CC2)cc(Cl)c2cccnc12',
       'c1ccc(-c2csc(N3CCN(c4ccccn4)CC3)n2)cc1',
       'c1ccc(C2=Nn3c(nnc3-c3cc(-c4ccccc4)n[nH]3)SC2)cc1'],
      dtype='object', name='SMILES', length=528)

Writing outcomes to CSV file

In [ ]:
df_untested=pd.read_csv('untested_molecules.csv')
for i in range(len(df_untested['SMILES'])):
    molecule=df_untested.loc[i,'SMILES']
    if molecule in intersect:
        df_untested.loc[i,selected_kinase]=1
        print(molecule)
    else:
        df_untested.loc[i,selected_kinase]=0

df_untested.set_index('SMILES', inplace=True)
df_untested.to_csv('predicted_molecules')

NameError: name 'intersect' is not defined